In [ ]:
# crucial for import API interface and loading data
ON_KAGGLE: bool = True

In [ ]:
if ON_KAGGLE:
    import sys
    sys.path.append('/kaggle/input/imports')
    from helper_functions import split_datetime
    from actpred_plot import plot_actual_vs_pred
    from data_preprocessing import merge_data, remove_col
    from feature_engineering import * # this is bad practice, call functions explicitly
else:
    import sys
    sys.path.append('../imports')
    from helper_functions import split_datetime
    from actpred_plot import plot_actual_vs_pred
    from data_preprocessing import merge_data, remove_col
    from feature_engineering import * # this is bad practice, call functions explicitly

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
if ON_KAGGLE:
    DATA_DIR = "/kaggle/input/predict-energy-behavior-of-prosumers/"
else:
    DATA_DIR = "../data/"

# Read CSVs and parse relevant date columns
train = pd.read_csv(DATA_DIR + "train.csv")
client = pd.read_csv(DATA_DIR + "client.csv")
historical_weather = pd.read_csv(DATA_DIR + "historical_weather.csv")
forecast_weather = pd.read_csv(DATA_DIR + "forecast_weather.csv")
electricity_prices = pd.read_csv(DATA_DIR + "electricity_prices.csv")
gas_prices = pd.read_csv(DATA_DIR + "gas_prices.csv")
weather_station_to_county_mapping = pd.read_csv(DATA_DIR + 'weather_station_to_county_mapping.csv')


In [ ]:
# We merge all DataFrames 
merged_df = merge_data(
    train, client, historical_weather, forecast_weather, 
    electricity_prices, gas_prices, weather_station_to_county_mapping
)

# Drop all non needed columns (ids and timestamps)
merged_df = remove_col(merged_df, drop_row_id=False)

## Feature engineering

In [ ]:
merged_df = add_daylight_col(merged_df)

merged_df = add_capacity_col(merged_df)

merged_df = basic_improvements(merged_df)

merged_df = add_shifted_target(merged_df)

# merged_df = add_public_holiday_col(merged_df)

# merged_df = add_school_holiday_col(merged_df)

In [ ]:
merged_df.columns

## Training & Model Building

In [ ]:
drop_columns = [
    'target', 'hours_ahead_forecast_weather',
    'row_id', 'data_block_id', 'prediction_unit_id', 
    'longitude_hist_weather', 'latitude_hist_weather',
    'longitude_forecast_weather', 'latitude_forecast_weather'
]

selected_fields = ['county', 'is_business', 'product_type', 'is_consumption',
       'eic_count_client',
       'surface_solar_radiation_downwards_forecast_weather',
       'total_precipitation_forecast_weather', 'year', 'week', 'hour',
       'day_of_year', 'day_of_week','daylight', 'capacity_per_eic',
       'squared_capacity_client', 'sum_column', 'temp_dew', 'shifted_target'
]

In [ ]:
X = merged_df.drop(['row_id', 'target'], axis=1)[selected_fields]
y = merged_df.target
                   

model = XGBRegressor(enable_categorical=True, max_depth=6, learning_rate=0.3)
model.fit(X, y)

# y_pred = bst.predict(X_test)

## main optimisation metric
# print('Mean absolute error test', mean_absolute_error(y_test, y_pred))
# print('Mean absolute error train', mean_absolute_error(y_train, bst.predict(X_train)))

In [ ]:
y_pred = model.predict(merged_df.drop(['row_id', 'target'], axis=1)[selected_fields])

# main optimisation metric
print('Mean absolute error', mean_absolute_error(merged_df.target, y_pred))


In [ ]:
if ON_KAGGLE:
    import enefit
else:
    import sys
    sys.path.append('../imports')
    import public_timeseries_testing_util as enefit


# copy of df before new data
merged_df['row_id'] = merged_df['row_id'].astype('int', errors='ignore')

env = enefit.make_env()
iter_test = env.iter_test()

counter = 0
previous_revealed_targets = pd.DataFrame()
all_revealed_targets = pd.DataFrame()

for (test, revealed_targets, client, historical_weather,
    forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:

    
    # if counter % 7 == 0:
    #     model.fit(merged_df.drop(['row_id', 'target'], axis=1)[selected_fields], merged_df.target)
        
    #print("Iteration #:", counter) 

    try: 
        # drop columns if target is na
        model_df = merged_df.dropna(subset=['target'])
    except:
        print('some na targets were dropped')
        # create alias anyway
        model_df = merged_df

    
    if counter in range(0,5):
        pass
        # print(f'Test dataframe #{counter} \n', test.head(3))
        # print(f'Revealed targets dataframe #{counter} \n', revealed_targets.head(3))
        # print(revealed_targets.columns)
        # print(f'Client dataframe #{counter} \n', client.head(3))
        # print(f'Historical weather dataframe #{counter} \n', historical_weather.head(3))
        # print(f'Forecast weather dataframe #{counter} \n', forecast_weather.head(3))
        # print(f'Electricity prices dataframe #{counter} \n', electricity_prices.head(3))
        # print(f'Gas prices dataframe #{counter} \n', gas_prices.head(3))
        # print(f'Sample prediction dataframe #{counter} \n', sample_prediction.head(3))
    
    prepped_df = merge_data(
        test, client, historical_weather, forecast_weather, 
        electricity_prices, gas_prices, weather_station_to_county_mapping
    )

    prepped_df = remove_col(prepped_df, drop_row_id=False)

    # rename the target column of the revealed targets for merging
    revealed_targets.rename(columns={'target' : 'shifted_target'}, inplace=True)
    # introduce a hour column to merge on the prepped df
    revealed_targets.datetime = pd.to_datetime(revealed_targets.datetime)
    revealed_targets = split_datetime(revealed_targets)
    # take only needed columns
    sel_revealed_targets = revealed_targets[['county', 'is_business', 'product_type', 'is_consumption','hour','shifted_target']]
    # merge the revealed targets as shifted target to the prepped_df
    prepped_df = pd.merge(prepped_df, sel_revealed_targets, on= ['county', 'is_business', 'product_type', 'is_consumption', 'hour'], how='left')

    # feature engineering
    prepped_df = add_daylight_col(prepped_df)
    prepped_df = add_capacity_col(prepped_df)
    prepped_df = basic_improvements(prepped_df)

    # prepped_df = add_public_holiday_col(prepped_df)
    # prepped_df = add_school_holiday_col(prepped_df)

    # print(merged_df.columns, '\n', prepped_df.columns)

    # pd.merge([merged_df, revealed_targets[['row_id', 'target']]], on=['row_id'], how='left', suffixes=('', '_revealed'))
    # merged_df['target'] = merged_df[['target', 'target_revealed']].apply(lambda x: x.to_list()[0] if x.to_list()[0] else x.to_list()[1], axis=1)
    # merged_df.drop('target_revealed', axis=1, inplace=True)
    # leave out for now: using revealed targets as additional feature
    # # bring new data to storage
    # merged_df = pd.concat([merged_df, prepped_df], axis=0, ignore_index=True)
    
    # try:
    #     # add revealed targets to data
    #     revealed_targets = pd.concat([previous_revealed_targets, revealed_targets], axis=0, ignore_index=True)
    #     targets_indexes = merged_df['row_id'][revealed_targets['row_id']].index
    #     merged_df['target'].iloc[targets_indexes] = revealed_targets['target']
    #     previous_revealed_targets = pd.DataFrame()
    # except KeyError as e:
    #     # store unused revealed targets for the next try
    #     print('KeyError occurred')
    #     print(e)
    #     previous_revealed_targets = revealed_targets.copy()

    sample_prediction['target'] = model.predict(prepped_df.drop('row_id', axis=1)[selected_fields])
    sample_prediction['target'] = sample_prediction['target'].fillna(0).clip(0)
    
    # send predictions
    env.predict(sample_prediction)    

    counter += 1
